In [ ]:
import tkinter
import random

DEFAULT_GAME = "______"

class OTGui:
    def __init__(self, master, fontsize=20, font='Arial', gamesfloc=None):
        
        self.init_data(gamesfloc)
        self.init_gui(master, fontsize=fontsize, font=font)
        
    def init_gui(self, master, fontsize, font):
        self.font = (font, fontsize)
        
        self.master = master
        master.title("OldTimer")
        tkinter.Label(self.master, font=self.font, text='Unplayed Games').grid(row=0,column=0)
        tkinter.Label(self.master, font=self.font, text='Match History').grid(row=2,column=0)
        self.unplayed_games = tkinter.Text(master, font=self.font)
        self.unplayed_games.grid(row=1,column=0)
        self.draw_game_list()
        self.player_frame = tkinter.Frame(master)
        self.player_frame.grid(row=0,column=1)        
        self.match_window = tkinter.Text(master, font=self.font)
        self.match_window.grid(row=3,column=0)

        self.player_frame = tkinter.Frame(master)
        self.player_frame.grid(row=1,column=1)
        tkinter.Label(self.player_frame,text='Player 1', font=self.font).grid(row=0,column=0)
        tkinter.Label(self.player_frame,text='Player 2', font=self.font).grid(row=0,column=1)
        self.player1 = tkinter.Entry(self.player_frame, font=self.font)
        self.player1.grid(row=1,column=0)
        self.player2 = tkinter.Entry(self.player_frame, font=self.font)
        self.player2.grid(row=1,column=1)
        tkinter.Button(self.player_frame, text="Roll!", font=self.font, command=self.roll).grid(row=2,column=0)
        self.chosen_game_window = tkinter.Label(self.player_frame, text=DEFAULT_GAME, font=self.font, relief="solid")
        self.chosen_game_window.grid(row=2,column=1)
        self.insert_match_button = tkinter.Button(self.player_frame, text="Lock in!", font=self.font, command=self.lock_in)
        self.insert_match_button.grid(row=3,column=1)
        tkinter.Button(self.player_frame, text="Reset!", font=self.font, command=self.reset).grid(row=3,column=0)
        tkinter.Button(self.player_frame, text="Reset & Reroll!", font=self.font, command=self.reset_and_roll).grid(row=4,column=0)

    def init_data(self,gamesfloc):
        self.match_history = []
        if(gamesfloc is None):
            self.games = []
        else:
            with open(gamesfloc) as f:
                self.games = f.readlines()
                self.games = [g[:-1] for g in self.games]
        self.current_game = ''
        
    def draw_game_list(self):
        self.unplayed_games.delete(1.0,tkinter.END)
        self.unplayed_games.insert(tkinter.INSERT,"\n".join(self.games))
        
    def draw_match_history(self):
        self.match_window.delete(1.0,tkinter.END)
        self.match_window.insert(tkinter.INSERT,"\n".join(self.match_history))
        
    def update_game_list(self):
        self.games = list(filter(None, self.unplayed_games.get("1.0",tkinter.END).split('\n')))
        
    def update_match_history(self):
        self.match_history = list(filter(None, self.match_window.get("1.0",tkinter.END).split('\n')))
        
    def upkeep(self):
        self.update_game_list()
        self.draw_game_list()
        self.update_match_history()
        self.draw_match_history()
        
    def roll(self):
        self.upkeep()
        if(not self.is_valid_players() or not self.is_default_game()):
            return
        random.shuffle(self.games)
        chosen_game = self.games.pop()
        self.draw_game_list()
        self.chosen_game_window['text'] = chosen_game
        return chosen_game
    
    def is_default_game(self):
        return self.chosen_game_window['text'] == DEFAULT_GAME
    
    def is_valid_players(self):
        return self.player1.get().strip() != '' and self.player2.get().strip() != '' 
    
    def is_invalid_state(self):
        return self.is_default_game() or not self.is_valid_players()
    
    def lock_in(self):
        self.upkeep()
        if(self.is_invalid_state()):
            return
        match = self.player1.get().strip() + "," + self.player2.get().strip() + "," + self.chosen_game_window.cget("text").strip()
        self.match_history.append(match+"\n")
        self.draw_match_history()
        self.player1.delete(0, tkinter.END)
        self.player2.delete(0, tkinter.END)
        self.chosen_game_window['text'] = DEFAULT_GAME
        
        
    def reset(self):Kevin
        self.upkeep()
        if(self.is_invalid_state()):
            return
        game = self.chosen_game_window['text']
        self.chosen_game_window['text'] = DEFAULT_GAME
        self.games.append(game+'\n')
        self.draw_game_list()
        return game
    
    def reset_and_roll(self):
        self.reset()
        self.roll()
        

floc = 'games.txt'
root = tkinter.Tk()
my_gui = OTGui(root,gamesfloc=floc)
root.mainloop()

In [3]:
with open(floc) as f:
    games = f.readlines()